In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-sales-analysis/Walmart_sales_Dashboard.pbix
/kaggle/input/walmart-sales-analysis/Walmart Sales Analysis And Marketing Recommendations.pdf
/kaggle/input/walmart-sales-analysis/Walmart_sales_analysis.csv


In [3]:
# Since you’re transitioning from accounting to data analytics and already learning SQL and Power BI, this is what I recommend:

# 🔧 Project Plan:
# Clean and explore the data in Python (optional if very clean).

df = pd.read_csv('/kaggle/input/walmart-sales-analysis/Walmart_sales_analysis.csv')
df.head


<bound method NDFrame.head of       Store_Number        Date Weekly_Sales  Holiday_Flag  Temperature  \
0                1    2/5/2010    1,643,691             0        42.31   
1                1   2/12/2010    1,641,957             1        38.51   
2                1   2/19/2010    1,611,968             0        39.93   
3                1   2/26/2010    1,409,728             0        46.63   
4                1    3/5/2010    1,554,807             0        46.50   
...            ...         ...          ...           ...          ...   
6430            45   9/28/2012      713,174             0        64.88   
6431            45   10/5/2012      733,455             0        64.89   
6432            45  10/12/2012      734,464             0        54.47   
6433            45  10/19/2012      718,126             0        56.47   
6434            45  10/26/2012      760,281             0        58.85   

      Fuel_Price   CPI   Unemployment  
0          2.572    211         8.106  
1

In [4]:
df.describe()

,Store_Number,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,0.069930,60.663782,3.358607,171.583994,7.999151
std,12.988182,0.255049,18.444933,0.459020,39.351305,1.875885
min,1.000000,0.000000,-2.060000,2.472000,126.000000,3.879000
25%,12.000000,0.000000,47.460000,2.933000,132.000000,6.891000
50%,23.000000,0.000000,62.670000,3.445000,183.000000,7.874000
75%,34.000000,0.000000,74.940000,3.735000,213.000000,8.622000
max,45.000000,1.000000,100.140000,4.468000,227.000000,14.313000


In [5]:
df.isnull().sum()

Store_Number    0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
 CPI            0
Unemployment    0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.dtypes

Store_Number      int64
Date             object
Weekly_Sales     object
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
 CPI              int64
Unemployment    float64
dtype: object

In [8]:
# Convert Date from text to date type.
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
# Load the CSV into SQL (use SQLite or PostgreSQL).

import sqlite3
conn = sqlite3.connect("walmart_cleaned.db")
df.to_sql('walmart_sales',conn, if_exists='replace', index=False)


from IPython.display import display
import pathlib

display(pathlib.Path("walmart_cleaned.db"))

PosixPath('walmart_cleaned.db')

In [10]:
# Write SQL queries to answer:
pd.read_sql_query("PRAGMA table_info(walmart_sales);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Store_Number,INTEGER,0,None,0
1,1,Date,TIMESTAMP,0,None,0
2,2,Weekly_Sales,TEXT,0,None,0
3,3,Holiday_Flag,INTEGER,0,None,0
4,4,Temperature,REAL,0,None,0
5,5,Fuel_Price,REAL,0,None,0
6,6,CPI,INTEGER,0,None,0
7,7,Unemployment,REAL,0,None,0


In [11]:
# Store-Specific Performance
pd.read_sql_query(
     """
     SELECT Store_Number, sum(Weekly_Sales) AS Revenue
     FROM walmart_sales
     GROUP BY Store_Number
     ORDER BY Revenue DESC
     """,
     conn
 )

,Store_Number,Revenue
0,8,114735.0
1,34,114340.0
2,17,109536.0
3,45,103574.0
4,21,100444.0
5,40,97027.0
6,25,95131.0
7,43,90497.0
8,35,87576.0
9,15,84335.0


In [12]:
# Analyze Holiday Impact on Sales per store
pd.read_sql_query(
    """
    SELECT Store_Number, 
    ROUND(AVG(CASE WHEN Holiday_flag = 1 THEN Weekly_Sales END),2) AS avg_holiday_sales,
    ROUND(AVG(CASE WHEN Holiday_flag = 0 THEN Weekly_Sales END),2) AS avg_nonholiday_sales,
    ROUND(((ROUND(AVG(CASE WHEN Holiday_flag = 1 THEN Weekly_Sales END),2)-ROUND(AVG(CASE WHEN Holiday_flag = 0 THEN Weekly_Sales END),2))/ROUND(AVG(CASE WHEN Holiday_flag = 0 THEN Weekly_Sales END),2))*100,2) AS sales_change_percentage
    FROM walmart_sales
    GROUP BY Store_Number
    ORDER BY sales_change_percentage DESC
    """,
    conn
)

,Store_Number,avg_holiday_sales,avg_nonholiday_sales,sales_change_percentage
0,32,96.4,1.00,9540.00
1,10,1.5,1.16,29.31
2,2,1.5,1.17,28.21
3,7,671.9,546.69,22.90
4,39,1.2,1.02,17.65
5,29,606.4,517.74,17.12
6,23,1.2,1.03,16.50
7,6,1.2,1.03,16.50
8,20,1.9,1.65,15.15
9,4,1.9,1.65,15.15


In [19]:
# Impact of high fuel prices
pd.read_sql_query(
    """
    SELECT
    (CASE 
        WHEN Fuel_Price < 2.5 THEN 'Very Cheap '
        WHEN Fuel_Price BETWEEN 2.5 AND 3 THEN 'Average'
        WHEN Fuel_Price BETWEEN 3 AND 3.5 THEN 'Moderate High'
        WHEN Fuel_Price BETWEEN 3.5 AND 4 THEN 'High'
        WHEN Fuel_Price >4 THEN 'Very High' END
            ) AS Fuel_Price_range,
    Count(*) AS number_of_weeks,
    AVG(Weekly_Sales) AS Avg_weekly_sales
    FROM walmart_sales
    GROUP BY Fuel_Price_range
    ORDER BY Avg_weekly_sales DESC
    """, 
    conn
)

,Fuel_Price_range,number_of_weeks,Avg_weekly_sales
0,Very Cheap,1,470.000000
1,High,2625,339.811429
2,Average,1872,316.299679
3,Moderate High,1533,304.255708
4,Very High,404,286.663366


In [ ]:

# Impact of high fuel prices (chart)
# Seasonal effects (chart)
# CPI vs. sales
# Analyze Economic Indicators' Impact  
# Business Recommendations Based on Insights
# Export aggregated tables into Power BI or Excel dashboards.
# Add business insights + a 1-page PDF report or Notion page summary.
# Top-performing weeks (Date was collected on same day_of_week)